### 如何调试梯度

In [2]:
import numpy as np 
import matplotlib.pyplot as plt

In [4]:
# 这里我们又产生了一些人工的不需要归一化的数据，模拟样本
np.random.seed(666)
X = np.random.random(size=(1000, 10)) # 1000个样本，10个特征，不知道怎么分布的

In [6]:
true_theta = np.arange(1, 12, dtype=float) # 可以包含小数步长，返回的是np.array类型

In [7]:
X_b = np.hstack([np.ones((len(X), 1)), X])
y = X_b.dot(true_theta) + np.random.normal(size=1000)

In [8]:
X.shape

(1000, 10)

In [10]:
y.shape # y本来就是标签向量，无所谓行列向量

(1000,)

In [11]:
true_theta

array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.])

In [14]:
def J(theta, X_b, y): # 这是一个连续函数，对向量而言的
    try:
        return np.sum((y - X_b.dot(theta))**2) / len(X_b) # 不看很好啊，完全没有影响啊
    except OverflowError:
        return float('inf')

In [15]:
# 数学推导的梯度的求法
def dJ_math(theta, X_b, y):
    return X_b.T.dot(X_b.dot(theta) - y) * 2. / len(y)

![](https://i.loli.net/2018/02/14/5a8417abcd52f.png)

In [16]:
# debug的梯度求法，可以复用的
def dJ_debug(theta, X_b, y, epsilon=0.01):
    # 这个是求梯度，求的是某个点的梯度，梯度含特征个数+1个元素
    # 一个梯度向量是对每一个求theta元素的偏导数
    res = np.empty(len(theta))
    for i in range(len(theta)): # 索引从0开始，对n+1个theta求偏导数，这么多年啥都没干
        theta_1 = theta.copy()
        theta_1[i] += epsilon
        theta_2 = theta.copy()
        theta_2[i] -= epsilon
        
        res[i] = (J(theta_1, X_b, y) - J(theta_2, X_b, y)) / (2*epsilon)
    return res # 一定要做有结果的事情

In [17]:
# 任何心理都是基于之前的经验，你不可能突变，想要突变必须经历某种东西
# 即使心理无形，也不可能你要变成什么就变什么，所以不要强迫或者否定自己的心理
# 只要知道你可以做什么来达到预期心理即可
def gradient_descent(dJ, X_b, y, initial_theta, eta, n_iters=1e4, epsilon=1e-8):
    theta = initial_theta
    cur_iter = 0
    
    while cur_iter < n_iters:
        gradient = dJ(theta, X_b, y)
        last_theta = theta
        theta = theta - eta * gradient # 沿梯度方向下降的意思
        if(abs(J(theta, X_b, y) - J(last_theta, X_b, y)) < epsilon):
            break
        
        cur_iter += 1
        
    return theta

In [19]:
X_b = np.hstack([np.ones((len(X), 1)), X])
initial_theta = np.zeros(X_b.shape[1])
eta = 0.01

In [21]:
# initial_theta是这个函数层别的输入参数
# 速度慢很多，可以先用这个作为正确结果，再求数学解
%time theta = gradient_descent(dJ_debug, X_b, y, initial_theta, eta)
theta 

Wall time: 4.72 s


array([  1.1251597 ,   2.05312521,   2.91522497,   4.11895968,
         5.05002117,   5.90494046,   6.97383745,   8.00088367,
         8.86213468,   9.98608331,  10.90529198])

In [24]:
%time theta = gradient_descent(dJ_math, X_b, y, initial_theta, eta)
theta

Wall time: 711 ms


array([  1.1251597 ,   2.05312521,   2.91522497,   4.11895968,
         5.05002117,   5.90494046,   6.97383745,   8.00088367,
         8.86213468,   9.98608331,  10.90529198])